In [1]:
import pandas as pd
import os
import time
import datetime

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%ls 

gdrive/  sample_data/


In [4]:
%cd gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP/

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.1 MB/s eta 0:00:00


In [6]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.8 MB/s eta 0:00:00


In [7]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, GPT2LMHeadModel
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
# import pytorch_lighting as pl
from sklearn.model_selection import train_test_split
import numpy as np
import random
import textwrap
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
tokenizer.add_special_tokens({
    'additional_special_tokens': [
      '</I>',
      '</GEN>',
    ]
})

2

In [10]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<|pad|>',
 'additional_special_tokens': ['</I>', '</GEN>']}

In [11]:
train_df = pd.read_csv('train_df.csv')

In [12]:
train_df.head()

,Unnamed: 0,title,ingred,instructions
0,0,Smoked Pork Butt,pounds fresh pork butt roast </I> tablespoons ...,if desired soak the pork butt in a brine solu...
1,1,Low Fat Fresh Peach Cobbler,ripe peaches skinned and sliced </I> tablespoo...,cinnamon and sugar for shaking on top. combin...
2,2,Mom's Hawaiian Wedding Cake,cans crushed pineapple </I> angel food cake ri...,in a 8x10 pan break up the angel food cake in...
3,3,Zesty Chicken Breasts,s lemons zest of grated </I> boneless skinless...,rub equal amounts of lemon zest into each sid...
4,4,Daily Intake of Vegetables Chili,ground beef </I> oil </I> lg onion chopped </I...,brown ground beef until completely brown. rin...


In [13]:
test_df = pd.read_csv('test_df.csv')

In [14]:
test_df.head()

,Unnamed: 0,title,ingred,instructions
0,0,Butternut Squash Lasagna,butternut squash diced about squash </I> tbsp ...,preheat oven to 450f. oil 2 large shallow bak...
1,1,Randy's Delicious Crock Pot Stuffing,butter </I> celery finely chopped </I> onion f...,for the bread around christmas thanksgiving e...
2,2,Wortel Salade - Carrot Salad,carrot peeled and chopped </I> celery diced </...,combine carrots celery and bell pepper. moist...
3,3,Good Fries,large russet potaes </I> salt </I> oil for dee...,peel the potatoes and cut into 3 8 inch 1 cm ...
4,4,Tiramisu,egg yolks </I> white sugar divided </I> pound ...,in a medium bowl beat together the egg yolks ...


In [15]:
train_df.ingred = "<|startoftext|> " + train_df.ingred + " </GEN> "
train_df.instructions = train_df.instructions + " <|endoftext|>"

In [16]:
test_df.ingred = "<|startoftext|> " + test_df.ingred + " </GEN> "
test_df.instructions = test_df.instructions + " <|endoftext|>"

In [17]:
test = train_df.iloc[30]
test

Unnamed: 0                                                     30
title                                          Maple Bacon Donuts
ingred          <|startoftext|> brown rice flour </I> white ri...
instructions     preheat the oven to 325 degrees f grease your...
Name: 30, dtype: object

In [18]:
sample_ip = tokenizer(test['ingred'])
input_id = sample_ip['input_ids']
sample_ip

{'input_ids': [50257, 7586, 11464, 10601, 50259, 11186, 11464, 10601, 50259, 65, 8158, 2266, 3939, 477, 4007, 21578, 5787, 16871, 10601, 50259, 79, 4265, 39515, 50259, 6018, 15763, 50259, 65, 868, 11913, 50259, 82, 16871, 20584, 50259, 82, 2124, 272, 14813, 27142, 50259, 82, 35652, 50259, 8899, 293, 24208, 50259, 66, 420, 16478, 3056, 24178, 50259, 83, 2977, 27575, 5576, 7277, 2945, 17180, 10746, 50259, 83, 2977, 27575, 16858, 7925, 50259, 8899, 293, 7925, 50259, 79, 322, 67, 1068, 7543, 50259, 13959, 392, 13959, 50259, 82, 31377, 7925, 50259, 82, 677, 274, 6546, 8968, 21385, 48898, 290, 23018, 1566, 23453, 50260], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenizer.decode(sample_ip['input_ids'])

'<|startoftext|>  brown rice flour </I> white rice flour </I> bobs red mill all purpose glutenfree baking flour </I> pota starch </I> arrowroot </I> baking powder </I> s baking soda </I> s xanthan gum </I> sugar </I> maple syrup </I> coconut oil melted </I> tablespoons unsweetened apple sauce </I> tablespoons vanilla extract </I> maple extract </I> powdered sugar </I> halfandhalf </I> s maple extract </I> slices thickcut bacon diced and fried until crisp </GEN>'

In [20]:
preds = [tokenizer.decode(input_id, skip_special_token = True, clean_up_tokenization_spaces=True) for input_id in sample_ip['input_ids'] ]

In [21]:
" ".join(preds)

'<|startoftext|>  brown  rice  flour </I> white  rice  flour </I> b obs  red  mill  all  purpose  gluten free  baking  flour </I> p ota  starch </I> arrow root </I> b aking  powder </I> s  baking  soda </I> s  x an than  gum </I> s ugar </I> map le  syrup </I> c oc onut  oil  melted </I> t ables poons  uns weet ened  apple  sauce </I> t ables poons  vanilla  extract </I> map le  extract </I> p ow d ered  sugar </I> half and half </I> s  maple  extract </I> s lic es  thick cut  bacon  diced  and  fried  until  crisp </GEN>'

In [22]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'pad_token': '<|pad|>',
 'additional_special_tokens': ['</I>', '</GEN>']}

In [23]:
tokenizer.bos_token_id, tokenizer.eos_token_id

(50257, 50256)

In [26]:
train_df.head()
# train_df.ingred

,Unnamed: 0,title,ingred,instructions
0,0,Smoked Pork Butt,<|startoftext|> pounds fresh pork butt roast <...,if desired soak the pork butt in a brine solu...
1,1,Low Fat Fresh Peach Cobbler,<|startoftext|> ripe peaches skinned and slice...,cinnamon and sugar for shaking on top. combin...
2,2,Mom's Hawaiian Wedding Cake,<|startoftext|> cans crushed pineapple </I> an...,in a 8x10 pan break up the angel food cake in...
3,3,Zesty Chicken Breasts,<|startoftext|> s lemons zest of grated </I> b...,rub equal amounts of lemon zest into each sid...
4,4,Daily Intake of Vegetables Chili,<|startoftext|> ground beef </I> oil </I> lg o...,brown ground beef until completely brown. rin...


In [27]:
class RecipeDataset(Dataset):
  def __init__(self, dataframe, tokenizer, source_len, summ_len, gpt2_type="gpt2"):
    self.dataframe = dataframe
    self.ingred = self.dataframe.ingred
    self.instructions = self.dataframe.instructions
    self.tokenizer = tokenizer
    self.gpt2_type = gpt2_type
    self.source_len = source_len
    self.summ_len = summ_len
    
  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, idx):
    text = str(self.ingred[idx])
    text = ' '.join(text.split())

    ctext = str(self.instructions[idx])
    ctext = ' '.join(ctext.split())

    source = self.tokenizer.batch_encode_plus([text], pad_to_max_length=True, max_length=self.source_len, return_tensors='pt', truncation=True)
    target = self.tokenizer.batch_encode_plus([ctext], pad_to_max_length=True, max_length=self.summ_len, return_tensors='pt', truncation=True)

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_mask.to(dtype=torch.long)
            }

In [28]:
train_params = {
        'batch_size': 2,
        'shuffle': True,
        'num_workers': 0
    }

val_params = {
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 0
    }


In [29]:
train_dataset = RecipeDataset(train_df, tokenizer, source_len=30, summ_len=150)

In [41]:
val_dataset = RecipeDataset(test_df.head(2000), tokenizer, source_len=30, summ_len=150)

In [42]:
train_dataloader = DataLoader(train_dataset, **train_params, drop_last=True)
val_dataloader = DataLoader(val_dataset,**val_params, drop_last=True)

## Fine Tuning GPT

In [43]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
# device = torch.device("cuda")
model.to(device)

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val) 

In [33]:
!nvidia-smi

Sat Apr 15 22:48:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   1141MiB / 15360MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [50]:
epochs = 3
learning_rate = 1e-5
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [51]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [54]:
def train(epoch, tokenizer, model, device, loader, optimizer, val_loader=None, model_class="t5", save_dir="/models"):
    model.train()
    batch_count = len(loader)
    for iteration, data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype=torch.long)
        mask = data['source_mask'].to(device, dtype=torch.long)

        outputs = model(input_ids=ids, attention_mask=mask,labels=ids)
        loss = outputs[0]

        if iteration % 500 == 0:
            batches_left = batch_count - iteration
            print(
                f'\nEpoch: {epoch}, Iteration: {iteration}, Loss:  {loss.item()}, Batches left: {batches_left}')

        if iteration % 5000 == 0:
            model.save_pretrained(save_dir + "/iter_{}_model".format(iteration))
            tokenizer.save_pretrained(save_dir + "/iter_{}_tokenizer".format(iteration))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if iteration % 1500 == 0 and val_loader != None:
            log_eval(epoch, tokenizer, model, device, val_loader, model_class=model_class)
            model.train()


def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    sources = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype=torch.long)
            ids = data['source_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                do_sample=True,
                max_length=int(os.environ['OUT_LEN']),
                num_beams=5,
                top_k=50,
                top_p=0.95
            )

            preds = [
                tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for
                g in generated_ids]
            target = [
                tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for
                t in y]
            source = [
                tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for
                s in ids]

            if _ % 100 == 0:
                print(f'Completed {_}')

            sources.extend(source)
            predictions.extend(preds)
            actuals.extend(target)
    return sources, predictions, actuals


def log_eval(epoch, tokenizer, model, device, loader, sample_limit=5000, model_class="t5"):
    model.eval()
    total_loss = 0
    loss_count = 0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            if model_class == "t5":
                outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
            else:
                outputs = model(input_ids = ids, attention_mask = mask, labels=ids)

            loss = outputs[0]
            total_loss += loss.item()
            loss_count += 1
    print("Eval Loss: {}".format(total_loss / loss_count))

In [ ]:
for epoch in range(epochs):
  train(epoch=epoch, 
        tokenizer=tokenizer,
        model=model,
        device=device, 
        loader=train_dataloader, 
        optimizer=optimizer, 
        val_loader=val_dataloader, 
        model_class="gpt2",
        save_dir="/models")
  
  model.save_pretrained('{}/checkpoint_{}'.format("/ep_models", epoch))
  tokenizer.save_pretrained('{}/checkpoint_{}'.format("/ep_models", epoch))
model.save_pretrained('/models')

0it [00:00, ?it/s]


Epoch: 0, Iteration: 0, Loss:  3.9703493118286133, Batches left: 360402


3it [00:35,  9.30s/it]

Eval Loss: 3.6342472119927405


503it [01:15, 13.13it/s]

...


Epoch: 0, Iteration: 12500, Loss:  2.6566076278686523, Batches left: 347902


13003it [22:33, 13.40it/s]


Epoch: 0, Iteration: 13000, Loss:  3.556729316711426, Batches left: 347402


13499it [23:13, 13.10it/s]


Epoch: 0, Iteration: 13500, Loss:  3.2575414180755615, Batches left: 346902


In [ ]:
for itr, data in enumerate(train_dataloader, 0):
  print(data)
  break

In [ ]:
from pandas.core.arrays.arrow.dtype import pa
pa